In [16]:
# importujemy wymagane biblioteki
from time import sleep
from datetime import datetime
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup

In [17]:
def download_page(page_number: int, job_name: str) -> BeautifulSoup:
    """
    Wchodzi na stronę oraz pobiera jej całkowitą zawartość
    """
    global browser
    url = 'https://nofluffjobs.com/pl/jobs/?criteria=keyword%3D"{job_name}"&page={page_number}'

    browser.get(url.format(
        job_name=job_name,
        page_number=page_number)
        )
    html = browser.page_source

    return BeautifulSoup(html)

In [18]:
def is_empty_page(page) -> bool:
    """
    Sprawdza czy strona jest pusta
    """
    empty_condition = page.find('nfj-no-offers-found-header')

    if empty_condition is None:
        return False
    else:
        return True

In [19]:
def get_job_offers(job_name: str) -> list:
    """
    Pobiera aktualnie dostępne oferty pracy dla danego zapytania

    @param job_name: zapytanie, które chcemy przeszukać
    @returns: lista ze słownikiem {page - numer strony, job - stanowisko, html - oryginalny html strony}
    """

    results = []  # zmienna informacjami o stronach
    page_number = 1

    # będziemy wykonywać pętle tak długo, aż nie trafimy na pustą stronę
    while True:  
        # pobieramy dane dla danego zawodu oraz strony
        bs = download_page(
            job_name=job_name,
            page_number=page_number)
        
        # sprawdzamy czy strona nie jest pusta
        if is_empty_page(bs):
            break

        # pobieramy informacje o ofertach pracy
        results.append({'page': page_number, 'job': job_name, 'html': str(bs)})

        page_number += 1  # przechodzimy do kolejnej strony
        sleep(5)  # czekamy 5 sekund przed kolejnym odpytaniem

    return results

In [20]:
jobs = ['data analyst', 'data scientist', 'data engineer']

In [21]:
browser = Chrome()
browser.implicitly_wait(10)

# otwórz stronę i kliknij na accept cookies
# odpal ją w maksymalnej rozdzielczości, bo strony są responsywne i nie wiadomo co się wyświetli na połowie ekranu
browser.get('https://nofluffjobs.com')
browser.maximize_window()
browser.find_element(By.ID, 'onetrust-accept-btn-handler').click()

# pobieramy dostępne informacje o zawodach
data = []
for job in jobs:
    data += get_job_offers(job)
browser.close()

In [13]:
dt = datetime.now().strftime("%Y%m%d_%H%M%S")
for d in data:
    with open(
        '../data/raw/{job_name}_{page}_{dt}.html'.format(
            job_name=d['job'], 
            page=d['page'], 
            dt=dt), 
        encoding='UTF-8', 
        mode='w') as f:
        f.write(d['html'])